In [1]:
from keras.src.models import Sequential
from keras.src.layers.convolutional.conv2d import Conv2D
from keras.src.layers.pooling.max_pooling2d import MaxPooling2D
from keras.src.layers.core.dense import Dense
from keras.src.layers.regularization.dropout import Dropout
from keras.src.layers.reshaping.flatten import Flatten
from keras.src.utils import to_categorical
from keras.src.utils.image_utils import load_img
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)


def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode="grayscale" )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


train_features = extract_features(train['image'])

print("working...")

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                    image     label
0       images/validation\angry\10052.jpg     angry
1       images/validation\angry\10065.jpg     angry
2       im

  0%|          | 0/28821 [00:00<?, ?it/s]

working...


In [2]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [3]:
x_train = train_features/255.0
x_test = test_features/255.0

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [6]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [7]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [9]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [10]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [11]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 171s 739ms/step - accuracy: 0.2317 - loss: 1.8404 - val_accuracy: 0.2593 - val_loss: 1.7979
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 146s 644ms/step - accuracy: 0.2651 - loss: 1.7784 - val_accuracy: 0.3616 - val_loss: 1.6381
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 142s 628ms/step - accuracy: 0.3486 - loss: 1.6378 - val_accuracy: 0.4537 - val_loss: 1.4542
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 154s 683ms/step - accuracy: 0.4218 - loss: 1.4911 - val_accuracy: 0.4449 - val_loss: 1.4355
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 158s 700ms/step - accuracy: 0.4487 - loss: 1.4337 - val_accuracy: 0.5096 - val_loss: 1.2903
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 144s 639ms/step - accuracy: 0.4693 - loss: 1.3749 - val_accuracy: 0.5204 - val_loss: 1.2573
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 144s 639ms/step - accuracy: 0.4876 - loss: 1.3480 - val_accuracy: 0.5276 - val_loss: 1.2494
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 158s 701ms/step - accuracy: 

In [13]:
model_json = model.to_json()
with open("emotiondetect.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetect.keras")